In [8]:
from google.colab import drive

# Google Drive 연결
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
file_path = '/content/drive/MyDrive/gdsc/train.xlsx'

import pandas as pd
# Excel 파일 불러오기
try:
    df = pd.read_excel(file_path)
    print(df.head())
except Exception as e:
    print(f"Excel 파일이 아님 혹은 에러: {e}")

  이주목적: 귀농,"우선순위: 교통, 문화, 의료시설","기타조건: 부모님과 함께","추천지역: 부산","이유: 귀농을 준비하면서도 부모님을 모셔야 하기에 교통·문화 인프라와 대형 병원이 가까운 부산이 적합합니다."
0  이주목적: 일자리,"우선순위: 교통, 문화, 자연환경","기타조건: 반려동물 동반"...                                                                     
1  이주목적: 삶의질,"우선순위: 교통, 문화, 물가","기타조건: 청년 창업 예정",...                                                                     
2  이주목적: 귀향및은퇴,"우선순위: 교통, 의료시설, 자연환경","기타조건: 아이들 ...                                                                     
3  이주목적: 경제적이유,"우선순위: 교통, 의료시설, 물가","기타조건: 차량 2대 ...                                                                     
4  이주목적: 직접 입력,"우선순위: 교통, 자연환경, 물가","기타조건: 소규모 자영...                                                                     


In [19]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [22]:
df.columns = ['full_text']
print(df.head())


                                           full_text
0  이주목적: 일자리,"우선순위: 교통, 문화, 자연환경","기타조건: 반려동물 동반"...
1  이주목적: 삶의질,"우선순위: 교통, 문화, 물가","기타조건: 청년 창업 예정",...
2  이주목적: 귀향및은퇴,"우선순위: 교통, 의료시설, 자연환경","기타조건: 아이들 ...
3  이주목적: 경제적이유,"우선순위: 교통, 의료시설, 물가","기타조건: 차량 2대 ...
4  이주목적: 직접 입력,"우선순위: 교통, 자연환경, 물가","기타조건: 소규모 자영...


In [24]:
from datasets import Dataset

# 각 행의 문자열을 list로 변환
texts = df["full_text"].astype(str).tolist()

dataset = Dataset.from_dict({"text": texts})
print(dataset)
# Dataset({
#     features: ['text'],
#     num_rows: N
# })



Dataset({
    features: ['text'],
    num_rows: 151
})


In [28]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

model_name = "skt/kogpt2-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

# 새로 PAD 토큰을 추가
special_tokens_dict = {'pad_token': '[PAD]'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

model = GPT2LMHeadModel.from_pretrained(model_name)
# 모델 토큰 임베딩 리사이즈 (토큰 개수 증가 반영)
model.resize_token_embeddings(len(tokenizer))


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(51201, 768)

In [29]:
from transformers import DataCollatorForLanguageModeling

MAX_LENGTH = 512

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        max_length=MAX_LENGTH,
        padding="max_length",
        truncation=True
    )

# Dataset.map()을 이용해 토크나이징
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# PyTorch 텐서 형태로 설정
tokenized_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask']
)

# 학습용 / 검증용 분리 (예: 90%:10%)
split_dataset = tokenized_dataset.train_test_split(
    test_size=0.1,
    seed=42
)
train_dataset = split_dataset["train"]
eval_dataset  = split_dataset["test"]

print("=== train_dataset ===")
print(train_dataset)
print("=== eval_dataset ===")
print(eval_dataset)

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

=== train_dataset ===
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 135
})
=== eval_dataset ===
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16
})


In [30]:
# GPT-계열 LM에서는 MLM=False
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [33]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./kogpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=7,              # 원하는 epoch 수
    per_device_train_batch_size=2,   # GPU 메모리에 맞게 조정
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",     # epoch마다 평가
    save_steps=200,
    save_total_limit=2,
    logging_steps=50,
    learning_rate=5e-5,
    load_best_model_at_end=False,    # 필요 시 True로
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.689200,1.362858
2,0.541400,1.425619
3,0.425700,1.400861
4,0.321500,1.436413
5,0.271900,1.469421
6,0.189300,1.479914
7,0.168600,1.482430


TrainOutput(global_step=476, training_loss=0.3561881710501278, metrics={'train_runtime': 200.6556, 'train_samples_per_second': 4.71, 'train_steps_per_second': 2.372, 'total_flos': 246920970240000.0, 'train_loss': 0.3561881710501278, 'epoch': 7.0})

In [35]:
trainer.save_model("/content/drive/MyDrive/gdsc/kogpt2-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/gdsc/kogpt2-finetuned")


('/content/drive/MyDrive/gdsc/kogpt2-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/gdsc/kogpt2-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/gdsc/kogpt2-finetuned/tokenizer.json')

In [77]:
from transformers import pipeline

inference_model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/gdsc/kogpt2-finetuned")
inference_tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/drive/MyDrive/gdsc/kogpt2-finetuned")

generator = pipeline(
    "text-generation",
    model=inference_model,
    tokenizer=inference_tokenizer
)

prompt = "이주목적: 경제적이유,우선순위: 의료시설, 자연환경, 물가 ,기타조건: 연로하신 부모님 부양"
output = generator(prompt, max_length=100, do_sample=True, top_k=50, top_p=0.9)
print(output[0]["generated_text"])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


이주목적: 경제적이유,우선순위: 의료시설, 자연환경, 물가 ,기타조건: 연로하신 부모님 부양","추천지역: 울산","이유: 울산은 태화강, 억새군락지 등 자연경관이 좋고, 의료 인프라·물가도 안정적이라 부모님 돌보며 일자리 찾기에 괜찮은 도시입니다."이유: 울산은 대규모 공원과 하천정비가 잘 되어 교육가정에 부담이 덜합니다."이유: 울산은 산업도시지만 공
